In [12]:
import pandas as pd
import requests as req
import numpy as np
from bs4 import BeautifulSoup

In [34]:
news = pd.read_csv("..\data\india_news_raw.csv")
news.head(5)

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic


In [35]:
keywords = ['adani','port business','cargo','ministry of ports','waterways',"mundra port", "krishnapatnam", "hazira",
"dhamra",
"dahej",
"vizag",
"mormugao",
"vizhinjam",
"kattupalli",
"kamarajar",
"tuna port",
"agardana",
"dighi"]
news = news[news['headline_text'].str.contains('|'.join(keywords))]
news['headline_text'] = news['headline_text'].str.lower()

In [36]:
news.reset_index(inplace=True)
news

,index,publish_date,headline_category,headline_text
0,6059,20010625,city.delhi,rs 8 cr cargo fraud detected at igi airport
1,17755,20010821,home.science,russian cargo ship heads to iss
2,29229,20010917,cricket,badani replaces injured laxman for sa tour
3,29285,20010917,cricket,badani ready to win his place back
4,36431,20011010,city.delhi,fewer cargo planes prolong elephant's stay
...,...,...,...,...
1168,3645637,20220321,city.kolkata,kol port's bangla link with ship-to-ship lpg c...
1169,3646740,20220323,city.ahmedabad,apsez handled 300mmt cargo
1170,3647704,20220325,city.hyderabad,air cargo saw massive surge in demand: industr...
1171,3647725,20220325,city.kolkata,bangladeshi cargo ship capsizes at kolkata port


In [11]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

# Download the required NLTK resources
nltk.download('vader_lexicon')

# Load the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Load the news dataset into a Pandas DataFrame
data = news

# Apply sentiment analysis to each headline and assign sentiment scores
data['NLTK'] = data['headline_text'].apply(lambda x: 1 if sia.polarity_scores(x)['compound'] > 0.1 else (-1 if sia.polarity_scores(x)['compound'] < -0.1 else 0))

# Extract the year from the publish_date column
data['year'] = data['publish_date'].astype(str).str[:4]

# Group the data by year and calculate the sum of sentiment scores for each year
yearly_sentiment_scores = data.groupby('year')['NLTK'].sum().to_dict()

# Display the sentiment scores year-wise
for year, sentiment_score in yearly_sentiment_scores.items():
    print(f"Year: {year}, Sentiment Score: {sentiment_score}")



[nltk_data] Error loading vader_lexicon: <urlopen error [WinError
[nltk_data]     10061] No connection could be made because the target
[nltk_data]     machine actively refused it>


Year: 2001, Sentiment Score: 1
Year: 2002, Sentiment Score: 1
Year: 2003, Sentiment Score: 3
Year: 2004, Sentiment Score: 2
Year: 2005, Sentiment Score: 0
Year: 2006, Sentiment Score: -2
Year: 2007, Sentiment Score: 3
Year: 2008, Sentiment Score: 9
Year: 2009, Sentiment Score: -5
Year: 2010, Sentiment Score: 1
Year: 2011, Sentiment Score: -9
Year: 2012, Sentiment Score: -3
Year: 2013, Sentiment Score: 4
Year: 2014, Sentiment Score: 2
Year: 2015, Sentiment Score: 3
Year: 2016, Sentiment Score: 10
Year: 2017, Sentiment Score: 4
Year: 2018, Sentiment Score: 15
Year: 2019, Sentiment Score: 2
Year: 2020, Sentiment Score: 0
Year: 2021, Sentiment Score: 9
Year: 2022, Sentiment Score: 8


In [16]:
#news = news.drop("sentiment_score", axis=1)
news

,publish_date,headline_category,headline_text,year,NLTK
0,20010720,city.ahmedabad,DRI seizes Adani HSD stock,2001,0
1,20011005,city.ahmedabad,I-T raids on shipping units,2001,0
2,20011029,business.india-business,India's PC shipment dips 5%,2001,0
3,20011030,city.mumbai,I-T raids offices of shipping agencies,2001,0
4,20011109,city.pune,First Divgi-Warner shipment leaves for China,2001,0
...,...,...,...,...,...
1329,20220323,city.ahmedabad,Adani Power inks MoU with IHI Corp; Korean firm,2022,0
1330,20220328,auto.cars,Adani Total forays into electric mobility infr...,2022,0
1331,20220328,city.ahmedabad,Adani Total Gas forays into EV segment,2022,0
1332,20220329,city.ahmedabad,Adani Group inks multi-year partnership with G...,2022,0


In [37]:
#news.to_csv("..\data\sentiment_news.csv")

BERT   

In [22]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the required NLTK resources
nltk.download('vader_lexicon')

# Load the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Load the news dataset into a Pandas DataFrame
data = news  # Assuming you have loaded the news dataset into the 'news' variable

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the sentiment classification model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.eval()  # Set the model to evaluation mode

# Tokenize and encode the headlines for sentiment analysis using BERT
encoded_inputs = tokenizer(data['headline_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Perform sentiment analysis using BERT
with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Assign sentiment scores based on BERT predictions
data['BERT_sentiment_score'] = predictions.tolist()

# Apply sentiment analysis to each headline and assign sentiment scores using SentimentIntensityAnalyzer
data['VADER_sentiment_score'] = data['headline_text'].apply(lambda x: 1 if sia.polarity_scores(x)['compound'] > 0.1 else (-1 if sia.polarity_scores(x)['compound'] < -0.1 else 0))

# Combine the sentiment scores from BERT and VADER to get the final sentiment score
data['final_sentiment_score'] = data['BERT_sentiment_score'] + data['VADER_sentiment_score']

# Extract the year from the publish_date column
data['year'] = data['publish_date'].astype(str).str[:4]

# Group the data by year and calculate the sum of final sentiment scores for each year
yearly_sentiment_scores = data.groupby('year')['final_sentiment_score'].sum().to_dict()

# Display the sentiment scores year-wise
for year, sentiment_score in yearly_sentiment_scores.items():
    print(f"Year: {year}, Sentiment Score: {sentiment_score}")



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

Year: 2001, Sentiment Score: 1
Year: 2002, Sentiment Score: 5
Year: 2003, Sentiment Score: 4
Year: 2004, Sentiment Score: 3
Year: 2005, Sentiment Score: 0
Year: 2006, Sentiment Score: -2
Year: 2007, Sentiment Score: 3
Year: 2008, Sentiment Score: 12
Year: 2009, Sentiment Score: -2
Year: 2010, Sentiment Score: 5
Year: 2011, Sentiment Score: -5
Year: 2012, Sentiment Score: 2
Year: 2013, Sentiment Score: 11
Year: 2014, Sentiment Score: 8
Year: 2015, Sentiment Score: 14
Year: 2016, Sentiment Score: 15
Year: 2017, Sentiment Score: 14
Year: 2018, Sentiment Score: 20
Year: 2019, Sentiment Score: 8
Year: 2020, Sentiment Score: 6
Year: 2021, Sentiment Score: 17
Year: 2022, Sentiment Score: 8


In [29]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Download the required NLTK resources
nltk.download('vader_lexicon')

# Load the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Load the news dataset into a Pandas DataFrame
data = news

# Apply sentiment analysis to each headline and assign sentiment scores
data['sentiment_score'] = data['headline_text'].apply(lambda x: 1 if sia.polarity_scores(x)['compound'] > 0.1 else (-1 if sia.polarity_scores(x)['compound'] < -0.1 else 0))

# Extract the year from the publish_date column
data['year'] = data['publish_date'].astype(str).str[:4]

# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['headline_text'])
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(data['headline_text'])
max_length = max(len(sequence) for sequence in sequences)
X = pad_sequences(sequences, maxlen=max_length)

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_length))
model.add(LSTM(128, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X, data['sentiment_score'], epochs=10, batch_size=32)

# Perform sentiment analysis using the trained LSTM model
predicted_probs = model.predict(X)
predicted_scores = [1 if prob > 0.5 else -1 for prob in predicted_probs]

# Assign sentiment scores based on predictions
data['LSTM_sentiment_score'] = predicted_scores

# Group the data by year and calculate the sum of sentiment scores for each year
yearly_sentiment_scores = data.groupby('year')['LSTM_sentiment_score'].sum().to_dict()

# Display the sentiment scores year-wise
for year, sentiment_score in yearly_sentiment_scores.items():
    print(f"Year: {year}, Sentiment Score: {sentiment_score}")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Epoch 1/10
42/42 [==============================] - 4s 23ms/step - loss: 0.6850 - accuracy: 0.5832
Epoch 2/10
42/42 [==============================] - 1s 22ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 3/10
42/42 [==============================] - 1s 24ms/step - loss: 0.6707 - accuracy: 0.5832
Epoch 4/10
42/42 [==============================] - 1s 25ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 5/10
42/42 [==============================] - 1s 23ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 6/10
42/42 [==============================] - 1s 22ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 7/10
42/42 [==============================] - 1s 19ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 8/10
42/42 [==============================] - 1s 21ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 9/10
42/42 [==============================] - 1s 20ms/step - loss: 0.6706 - accuracy: 0.5832
Epoch 10/10
42/42 [==============================] - 1s 11ms/step
Year: 2001, Sentiment Score: -6
Year: 2002,

In [1]:
news

NameError: name 'news' is not defined